<a href="https://colab.research.google.com/github/ajw1587/Pytorch_Study/blob/main/13_Overfitting_Dropout_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install torch
!pip install torchvision

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

dropout_prob = 0.5
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [43]:
train_dataset = dsets.MNIST(root = '../data/',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)
test_dataset = dsets.MNIST(root = '../data/',
                           train = False,
                           transform = transforms.ToTensor(),
                           download = True)

print(type(train_dataset))
print(train_dataset.data.shape)
print(type(test_dataset))
print(test_dataset.data.shape)

<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([60000, 28, 28])
<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([10000, 28, 28])


In [44]:
data_loader = torch.utils.data.DataLoader(train_dataset,
                                          100,
                                          shuffle = True,
                                          drop_last = True)

In [45]:
# Model
class MNIST_model(nn.Module):
  def __init__(self):
    super(MNIST_model, self).__init__()
    self.linear1 = nn.Linear(28 * 28, 100)
    self.linear2 = nn.Linear(100, 200)
    self.linear3 = nn.Linear(200, 100)
    self.linear4 = nn.Linear(100, 10)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(dropout_prob)

    torch.nn.init.xavier_uniform_(self.linear1.weight)
    torch.nn.init.xavier_uniform_(self.linear2.weight)
    torch.nn.init.xavier_uniform_(self.linear3.weight)
    torch.nn.init.xavier_uniform_(self.linear4.weight)

  def forward(self, x):
    x = self.linear1(x)
    x = self.relu(x)
    x = self.dropout(x)
    
    x = self.linear2(x)
    x = self.relu(x)
    x = self.dropout(x)

    x = self.linear3(x)
    x = self.relu(x)
    x = self.dropout(x)

    y = self.linear4(x)

    return y

In [46]:
model = MNIST_model().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [47]:
total_batch = len(data_loader)
print(total_batch)

600


In [48]:
for epoch in range(5):
  avg_cost = 0

  for X,Y in data_loader:
    x = X.view(-1, 28 * 28).to(device)
    y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(x)

    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('[Epoch: {}] cost = {}'.format(epoch, avg_cost))
print('Learning Finished')

[Epoch: 0] cost = 0.8720080852508545
[Epoch: 1] cost = 0.7383248209953308
[Epoch: 2] cost = 0.7127805948257446
[Epoch: 3] cost = 0.687138020992279
[Epoch: 4] cost = 0.697748064994812
Learning Finished
